# Sentiment Analysis

In [14]:

# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [21]:
 #This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.

In [22]:
data = pd.read_csv('../datasets/sentiment_analysis/Sentiment.csv')

In [23]:
data = data[['sentiment', 'text']]

In [24]:
data.head()

,sentiment,text
0,Neutral,RT @NancyLeeGrahn: How did everyone feel about...
1,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...
2,Neutral,RT @TJMShow: No mention of Tamir Rice and the ...
3,Positive,RT @RobGeorge: That Carly Fiorina is trending ...
4,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...


In [26]:
from keras_preprocessing.sequence import pad_sequences
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size) #Positive
print(data[ data['sentiment'] == 'Negative'].size) #Negative

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [28]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 28, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [30]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 40, batch_size=batch_size, verbose = 2)

Epoch 1/40
225/225 - 14s - loss: 0.4406 - accuracy: 0.8134 - 14s/epoch - 62ms/step
Epoch 2/40
225/225 - 12s - loss: 0.3165 - accuracy: 0.8670 - 12s/epoch - 51ms/step
Epoch 3/40
225/225 - 14s - loss: 0.2759 - accuracy: 0.8862 - 14s/epoch - 63ms/step
Epoch 4/40
225/225 - 16s - loss: 0.2467 - accuracy: 0.8987 - 16s/epoch - 70ms/step
Epoch 5/40
225/225 - 15s - loss: 0.2174 - accuracy: 0.9135 - 15s/epoch - 65ms/step
Epoch 6/40
225/225 - 15s - loss: 0.1984 - accuracy: 0.9210 - 15s/epoch - 66ms/step
Epoch 7/40
225/225 - 16s - loss: 0.1775 - accuracy: 0.9267 - 16s/epoch - 71ms/step
Epoch 8/40
225/225 - 17s - loss: 0.1570 - accuracy: 0.9366 - 17s/epoch - 75ms/step
Epoch 9/40
225/225 - 16s - loss: 0.1457 - accuracy: 0.9413 - 16s/epoch - 73ms/step
Epoch 10/40
225/225 - 15s - loss: 0.1341 - accuracy: 0.9467 - 15s/epoch - 68ms/step
Epoch 11/40
225/225 - 16s - loss: 0.1261 - accuracy: 0.9492 - 16s/epoch - 71ms/step
Epoch 12/40
225/225 - 16s - loss: 0.1225 - accuracy: 0.9496 - 16s/epoch - 73ms/step
E

In [31]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

64/64 - 1s - loss: 1.1195 - accuracy: 0.8221 - 771ms/epoch - 12ms/step
score: 1.12
acc: 0.82


In [32]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 210ms/epoch - 210ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 24ms/epoch - 24ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 24ms/epoch - 24ms/step
1/1 - 0s - 26ms/epoch - 26ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 24ms/epoch - 24ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 22ms/epoch - 22ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 24ms/epoch - 24ms/step
1/1 - 0s - 2

In [34]:
twt = ['bad']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 344]]
1/1 - 0s - 28ms/epoch - 28ms/step
negative


In [35]:
df=pd.read_csv('../csv/yes_whatabout_with_labels.csv')

In [56]:
# Load the pre-trained tokenizer and model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'])

pr=[]

# Predict the sentiment of each tweet in the new columns
for tweet in df['Text']:
    # Convert the tweet into a vector using the tokenizer
    twt = tokenizer.texts_to_sequences([tweet])

    # Pad the tweet to have the same shape as the input layer
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)

    # Predict the sentiment of the tweet using the model
    sentiment = model.predict(twt, batch_size=1, verbose=2)[0]

    # Determine the sentiment and print the result
    if np.argmax(sentiment) == 0:
        print(f"New tweet: {tweet} - sentiment: negative")
        pr.append('negative')
        
    elif np.argmax(sentiment) == 1:
        print(f"New tweet: {tweet} - sentiment: positive")
        pr.append('positive')


1/1 - 0s - 33ms/epoch - 33ms/step
New tweet: Urgent action to prevent a pandemic is of course necessary and pressing. But the climate crisis represents a far graver and deadlier existential threat, and yet the same sense of urgency is absent. - sentiment: negative
1/1 - 0s - 33ms/epoch - 33ms/step
New tweet: Dad: Billy, when did you last tidy your room? It’s an absolute tip. I’d like you to tidy it before you go to football practice please. Billy: That’s not fair! What about Bobby? His room is just as bad as mine and you’ve let him go out to his friend’s. You’re so unreasonable, it’s always one rule for him and another for me. I’m not doing it. - sentiment: negative
1/1 - 0s - 30ms/epoch - 30ms/step
New tweet: The West has no right to criticize our record on human rights, look at US actions in Central America, the history of slavery and of lynchings, not to mention apartheid in South Africa…. - sentiment: negative
1/1 - 0s - 31ms/epoch - 31ms/step
New tweet: but what about the economy?

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding/embedding_lookup' defined at (most recent call last):
    File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\lored\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\lored\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\lored\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\lored\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Prog